# Rolling Forecast:

In [ ]:
import os
import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from keras.models import load_model
from keras import backend as K

import pytz
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from tabulate import tabulate

from lstm_load_forecasting import data, lstm

from ast import literal_eval

from dateutil.tz import tzutc

%matplotlib

In [2]:
timezone = pytz.timezone('Europe/Zurich')
starting = timezone.localize(datetime.datetime(2017,2,1,0,0,0,0))
path = os.path.join('Data', 'fulldataset.csv')
res_path = os.path.abspath('results/')
model_dir = os.path.abspath('models/')
date = '20170508'

# Model 3:

In [3]:
df3 = data.load_dataset(path=path, modules=['actual', 'calendar'])
df3_scaled = df3.copy()
df3_scaled = df3_scaled.dropna()

# Get all float type columns
floats = [key for key in dict(df3_scaled.dtypes) if dict(df3_scaled.dtypes)[key] in ['float64']]
scaler = StandardScaler()
scaled_columns = scaler.fit_transform(df3_scaled[floats])
df3_scaled[floats] = scaled_columns

df3_train = df3_scaled.loc[(df3_scaled.index < starting )].copy()
#df3_train = df3_scaled.loc[(df3_scaled.index < starting) & (df3_scaled.index > starting - pd.DateOffset(months=15))].copy()
df3_test = df3_scaled.loc[df3_scaled.index >= starting].copy()
y_train = df3_train['actual'].copy()
X_train = df3_train.drop('actual', 1).copy()
y_test = df3_test['actual'].copy()
X_test = df3_test.drop('actual', 1).copy()

valid_results_3 = pd.read_csv(os.path.join(res_path, 'notebook_03/', str('model3_results_' + date + '.csv')), delimiter=';')
test_results_3 = pd.read_csv(os.path.join(res_path, 'notebook_03/', str('model3_test_results' + date + '.csv')), delimiter=';')
test_results_3 = test_results_3.sort_values('Mean absolute error', ascending=True)
best_model_3 = test_results_3.loc[0]['Model name']

config = valid_results_3.loc[valid_results_3['model_name'] == best_model_3]
batch_size = int(config['batch_train'].values[0])
size = int(y_test.shape[0] / batch_size)

layers = literal_eval(config['config'].values[0])
layers = layers['layers']

### First training of network

In [4]:
#batch_size = config['batch_train'].values 
batch_size = 8
model3 = lstm.create_model(layers=layers, sample_size=X_train.shape[0], batch_size=batch_size, timesteps=1, 
                           features=X_train.shape[1], loss='mse', optimizer='adam')
history = lstm.train_model(model=model3, mode='fit', y=y_train, X=X_train, 
                           batch_size=batch_size, timesteps=1, epochs=25, 
                           rearrange=False, validation_split=0.2, verbose=1)

Train on 14592 samples, validate on 3648 samples
Epoch 1/25
14592/14592 [==============================] - 8s - loss: 0.6626 - mean_absolute_error: 0.5513 - val_loss: 0.3794 - val_mean_absolute_error: 0.4876
Epoch 2/25
14592/14592 [==============================] - 8s - loss: 0.4613 - mean_absolute_error: 0.4324 - val_loss: 0.3131 - val_mean_absolute_error: 0.4640
Epoch 3/25
14592/14592 [==============================] - 8s - loss: 0.4353 - mean_absolute_error: 0.4127 - val_loss: 0.3139 - val_mean_absolute_error: 0.4638
Epoch 4/25
14592/14592 [==============================] - 8s - loss: 0.4217 - mean_absolute_error: 0.4018 - val_loss: 0.3214 - val_mean_absolute_error: 0.4630
Epoch 5/25
14592/14592 [==============================] - 8s - loss: 0.4124 - mean_absolute_error: 0.3947 - val_loss: 0.3197 - val_mean_absolute_error: 0.4595
Epoch 00004: early stopping


### Loop through windows 

In [5]:
starting_loc = starting
starting_utc = starting.astimezone(pytz.utc)
ending_utc = df3_test.tail(1).index[0]
ending_loc = ending_utc.tz_convert('Europe/Zurich')
day_changes = pd.date_range(start=starting_loc, end=ending_loc, freq='24h', tz="Europe/Zurich")

predictions = pd.DataFrame(index=pd.date_range(start=starting_loc, end=ending_loc, normalize=True, freq='60min', tz='Europe/Zurich'))
predictions.index = predictions.index.tz_convert('utc')

for idx, hour in enumerate(day_changes):
    if idx == 0:
        pass
    else:
        # Select the actual values from the previous day and retrain the model with that new data
        dh = df3_test.loc[(df3_test.index >= hour - pd.DateOffset(days=1)) & (df3_test.index < hour)]
        y_train = dh['actual']
        X_train = dh.drop('actual', 1)
        history = lstm.train_model(model=model3, mode='fit', y=y_train, X=X_train, 
                                   batch_size=batch_size, timesteps=1, epochs=13, rearrange=False, validation_split=0,
                                   verbose=0, early_stopping=False)
    # Select the input data for the next 2 days and create multistep forecast
    df = df3_test.loc[(df3_test.index >= hour) & (df3_test.index < hour + pd.DateOffset(days=2))]
    df = df.drop('actual', 1)
    X_predict = df
    scaled_predictions = lstm.get_predictions(model=model3, X=X_predict, batch_size=batch_size, timesteps=1, verbose=1)
    hour_utc = hour.tz_convert('utc')
    # Combine and store results
    window = pd.date_range(start=hour_utc, periods=len(scaled_predictions), freq='60min', tz='UTC')
    result = pd.DataFrame(data={"model3": scaled_predictions.flatten()}, index=window)
    predictions = predictions.combine_first(result)

mu = scaler.mean_[0]
sigma = scaler.scale_[0]
model3_predictions = mu + sigma*predictions

 8/48 [====>.........................] - ETA: 0sWarnining: Division "sample_size/batch_size" not a natural number.
Dropped the last 7 of 47 number of obs.
 8/40 [=====>........................] - ETA: 0sWarnining: Division "sample_size/batch_size" not a natural number.
Dropped the last 7 of 47 number of obs.
 8/24 [=========>....................] - ETA: 0s

# Model 6

In [6]:
df6 = data.load_dataset(path=path, modules=['all'])
df6_scaled = df6.copy()
df6_scaled = df6_scaled.dropna()

# Get all float type columns
floats = [key for key in dict(df6_scaled.dtypes) if dict(df6_scaled.dtypes)[key] in ['float64']]
scaler = StandardScaler()
scaled_columns = scaler.fit_transform(df6_scaled[floats])
df6_scaled[floats] = scaled_columns

df6_train = df6_scaled.loc[(df6_scaled.index < starting )].copy()
#df6_train = df6_scaled.loc[(df6_scaled.index < starting) & (df6_scaled.index > starting - pd.DateOffset(months=15))].copy()
df6_test = df6_scaled.loc[df6_scaled.index >= starting].copy()
y_train = df6_train['actual'].copy()
X_train = df6_train.drop('actual', 1).copy()
y_test = df6_test['actual'].copy()
X_test = df6_test.drop('actual', 1).copy()

valid_results_6 = pd.read_csv(os.path.join(res_path, 'notebook_06/', str('model6_results_' + date + '.csv')), delimiter=';')
test_results_6 = pd.read_csv(os.path.join(res_path, 'notebook_06/', str('model6_test_results' + date + '.csv')), delimiter=';')
test_results_6 = test_results_6.sort_values('Mean absolute error', ascending=True)
best_model_6 = test_results_6.loc[0]['Model name']

config = valid_results_6.loc[valid_results_6['model_name'] == best_model_6]
batch_size = int(config['batch_train'].values[0])
size = int(y_test.shape[0] / batch_size)

layers = literal_eval(config['config'].values[0])
layers = layers['layers']

In [7]:
batch_size = 8
model6 = lstm.create_model(layers=layers, sample_size=X_train.shape[0], batch_size=batch_size, timesteps=1, 
                           features=X_train.shape[1], loss='mse', optimizer='adam')
history = lstm.train_model(model=model6, mode='fit', y=y_train, X=X_train, 
                           batch_size=batch_size, timesteps=1, epochs=25, 
                           rearrange=False, validation_split=0.2, verbose=1)

Train on 14592 samples, validate on 3648 samples
Epoch 1/25
14592/14592 [==============================] - 22s - loss: 0.6832 - mean_absolute_error: 0.5695 - val_loss: 0.3487 - val_mean_absolute_error: 0.4661
Epoch 2/25
14592/14592 [==============================] - 21s - loss: 0.4484 - mean_absolute_error: 0.4183 - val_loss: 0.3439 - val_mean_absolute_error: 0.4239
Epoch 3/25
14592/14592 [==============================] - 22s - loss: 0.4206 - mean_absolute_error: 0.3994 - val_loss: 0.2205 - val_mean_absolute_error: 0.3639
Epoch 4/25
14592/14592 [==============================] - 22s - loss: 0.4065 - mean_absolute_error: 0.3884 - val_loss: 0.2317 - val_mean_absolute_error: 0.3755
Epoch 5/25
14592/14592 [==============================] - 22s - loss: 0.3991 - mean_absolute_error: 0.3799 - val_loss: 0.2402 - val_mean_absolute_error: 0.3834
Epoch 6/25
14592/14592 [==============================] - 20s - loss: 0.3893 - mean_absolute_error: 0.3706 - val_loss: 0.1915 - val_mean_absolute_error

In [8]:
starting_loc = starting
starting_utc = starting.astimezone(pytz.utc)
ending_utc = df6_test.tail(1).index[0]
ending_loc = ending_utc.tz_convert('Europe/Zurich')
day_changes = pd.date_range(start=starting_loc, end=ending_loc, freq='24h', tz="Europe/Zurich")

predictions = pd.DataFrame(index=pd.date_range(start=starting_loc, end=ending_loc, normalize=True, freq='60min', tz='Europe/Zurich'))
predictions.index = predictions.index.tz_convert('utc')

for idx, hour in enumerate(day_changes):
    if idx == 0:
        pass
    else:
        dh = df6_test.loc[(df6_test.index >= hour - pd.DateOffset(days=1)) & (df6_test.index < hour)]
        y_train = dh['actual']
        X_train = dh.drop('actual', 1)
        history = lstm.train_model(model=model6, mode='fit', y=y_train, X=X_train, 
                                   batch_size=batch_size, timesteps=1, epochs=13, rearrange=False, validation_split=0,
                                   verbose=0, early_stopping=False)
    
    df = df6_test.loc[(df6_test.index >= hour) & (df6_test.index < hour + pd.DateOffset(days=2))]
    df = df.drop('actual', 1)
    X_predict = df
    scaled_predictions = lstm.get_predictions(model=model6, X=X_predict, batch_size=batch_size, timesteps=1, verbose=1)
    hour_utc = hour.tz_convert('utc')
    window = pd.date_range(start=hour_utc, periods=len(scaled_predictions), freq='60min', tz='UTC')
    result = pd.DataFrame(data={"model6": scaled_predictions.flatten()}, index=window)
    predictions = predictions.combine_first(result)

mu = scaler.mean_[0]
sigma = scaler.scale_[0]
model6_predictions = mu + sigma*predictions



 8/48 [====>.........................] - ETA: 0sWarnining: Division "sample_size/batch_size" not a natural number.
Dropped the last 7 of 47 number of obs.
 8/40 [=====>........................] - ETA: 0sWarnining: Division "sample_size/batch_size" not a natural number.
Dropped the last 7 of 47 number of obs.
 8/24 [=========>....................] - ETA: 0s

# Rolling ARMA forecast

In [9]:
timezone = pytz.timezone('Europe/Zurich')
starting = timezone.localize(datetime.datetime(2017,2,1,0,0,0,0))
arma_fc = pd.read_csv(os.path.join('Data', 'arma_rolling_fc.csv'))
arma_forecasts = pd.DataFrame(data={"arma_forecast": arma_fc['x'].values}, index=pd.date_range(starting, periods=arma_fc.shape[0], freq='60min'))

# Results

In [13]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
dfc = data.load_dataset(path=path, modules=['actual', 'entsoe'])
forecasts = model3_predictions
forecasts = forecasts.join(model6_predictions)
forecasts = forecasts.join(dfc)
forecasts = forecasts.join(arma_forecasts)
forecasts = forecasts.dropna()
results = {}
results[''] = ['MSE', 'MAE', 'MAPE']
results['entsoe'] = [mean_squared_error(forecasts['actual'], forecasts['entsoe']), 
                     mean_absolute_error(forecasts['actual'], forecasts['entsoe']),
                     mean_absolute_percentage_error(forecasts['actual'], forecasts['entsoe'])
                    ]
results['arima'] = [mean_squared_error(forecasts['actual'], forecasts['arma_forecast']), 
                          mean_absolute_error(forecasts['actual'], forecasts['arma_forecast']),
                          mean_absolute_percentage_error(forecasts['actual'], forecasts['arma_forecast'])
                         ]
results['m6-all'] = [mean_squared_error(forecasts['actual'], forecasts['model6']), 
                     mean_absolute_error(forecasts['actual'], forecasts['model6']),
                     mean_absolute_percentage_error(forecasts['actual'], forecasts['model6'])
                    ]
results['m3-calendar'] = [mean_squared_error(forecasts['actual'], forecasts['model3']), 
                          mean_absolute_error(forecasts['actual'], forecasts['model3']),
                          mean_absolute_percentage_error(forecasts['actual'], forecasts['model3'])
                         ]
print(tabulate(results, headers='keys', numalign="right", tablefmt='latex', floatfmt=".1f"))

\begin{tabular}{lrrrr}
\hline
      &   entsoe &    arima &   m6-all &   m3-calendar \\
\hline
 MSE  & 426230.2 & 217798.2 & 127505.3 &      151625.5 \\
 MAE  &    516.9 &    373.6 &    273.7 &         302.7 \\
 MAPE &      7.7 &      5.5 &      4.1 &           4.5 \\
\hline
\end{tabular}


In [11]:
plt.figure()
plt.plot(forecasts.index, forecasts['entsoe'], label='ENTSOE Forecast')
plt.plot(forecasts.index, forecasts['actual'], label='Actual Load')
plt.plot(forecasts.index, forecasts['model6'], label='Model 6 (All)')
plt.plot(forecasts.index, forecasts['model3'], label='Model 3 (Calendar)')
plt.plot(forecasts.index, forecasts['arma_forecast'], label='ARIMA (0,1,5)')
plt.title('Forecast Comparison: Rolling Forecast')
plt.ylabel('Electricity load (in MW)')
plt.xlabel('Date')
plt.legend(loc='upper left')
plt.show

<function matplotlib.pyplot.show>

In [ ]:
K.clear_session()
import tensorflow as tf
tf.reset_default_graph()